In [3]:
import netCDF4
import numpy as np
import numpy.ma as ma

In [4]:
anzda_recon_txt = np.loadtxt('/d4/yzhou/Drought_atlas/ANZDA/anzda-recon.txt',skiprows=1)

In [5]:
time_np = anzda_recon_txt[:,0]

In [6]:
# delete time from anzda_recon_txt, which is the first element of each row
anzda_recon_txt = [row[1:] for row in anzda_recon_txt]

In [7]:
anzda_pdsi_lon_lat_pair_txt = np.loadtxt('/d4/yzhou/Drought_atlas/ANZDA/anzda-pdsi-xy.txt',usecols=(0,1))

In [8]:
lat_np = np.arange(-46.75,-11,0.5)

In [9]:
lon_np = np.arange(136.25,178.5,0.5)

In [10]:
pdsi_np = np.zeros((len(time_np), len(lon_np), len(lat_np)))

In [11]:
pdsi_ma = ma.masked_equal(pdsi_np,0)

In [12]:
anzda_recon_np = np.array(anzda_recon_txt)
for i in range(len(anzda_pdsi_lon_lat_pair_txt)):
    lon_txt = anzda_pdsi_lon_lat_pair_txt[i][0]
    lat_txt = anzda_pdsi_lon_lat_pair_txt[i][1]
    lon_index = np.where(lon_np == lon_txt)[0][0]
    lat_index = np.where(lat_np == lat_txt)[0][0]
    pdsi_ma[:,lon_index,lat_index] = anzda_recon_np[:,i]

In [13]:
nc = netCDF4.Dataset('/d4/yzhou/Drought_atlas/ANZDA/anzda.nc','w')

In [14]:
nc.createDimension('time', size=len(time_np))

<type 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 513

In [15]:
nc.createDimension('longitude', size=len(lon_np))

<type 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 85

In [16]:
nc.createDimension('latitude', size=len(lat_np))

<type 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 72

In [17]:
times = nc.createVariable("time","f8",("time",))
latitudes = nc.createVariable("latitude","f4",("latitude",))
longitudes = nc.createVariable("longitude","f4",("longitude",))

In [18]:
pdsi = nc.createVariable("pdsi", "f4", ("time", "longitude", "latitude",))

In [27]:
import time
import cf_units
nc.title = "Eastern Australia and New Zealand Drought Atlas (ANZDA)"
nc.history = "Original data downloaded from https://www.ncdc.noaa.gov/cdo/f?p=519:1:0::::P1_STUDY_ID:20245. Created " + time.ctime(time.time())
nc.file_author = "Yuxin Zhou yzhou@ldeo.columbia.edu"
nc.reference = "Jonathan G. Palmer, Edward R. Cook, Chris S.M. Turney, Kathy Allen, Pavla Fenwick, Benjamin I. Cook, Alison O'Donnell, Janice Lough, Pauline Grierson, Patrick Baker. 2015. Drought variability in the eastern Australia and New Zealand summer drought atlas (ANZDA, CE 1500-2012) modulated by the Interdecadal Pacific Oscillation. Environmental Research Letters, 10(12). doi: 10.1088/1748-9326/10/12/124002"
latitudes.units = "degrees north"
latitudes.long_name = "latitude"
latitudes.axis = "Y"
longitudes.units = "degrees east"
longitudes.long_name = "longitude"
longitudes.axis = "X"
pdsi.units = cf_units.Unit('1').format()
pdsi.long_name = "Palmer Drought Severity Index DJF"
pdsi._FillValue = pdsi.fill_value
time.units = "days since 0001-01-01 00:00:00.0"
time.calendar = "gregorian"
time.axis = "T"

In [28]:
pdsi_.fill_value

AttributeError: NetCDF: Attribute not found